In [1]:
import seaborn as sns
import metapack as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()


In [2]:
#pkg = mp.jupyter.open_package()
pkg = mp.jupyter.open_source_package()
pkg

# Monthly Retail Trade Report
`census.gov-monthly_retail-1.1.1` Last Update: 2020-05-19T16:07:31

_Retail and Food Service sales volume, by months, since 1992_


## Documentation Links

* [homepage](https://www.census.gov/retail/index.html) Dataset home page

 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)

## Resources

* ** [marts_adjusted](notebooks/Extract.ipynb#adj)**. Monthly retail sales, adjusted
* ** [marts_non_adjusted](notebooks/Extract.ipynb#nadj)**. Monthly retail sales, non adjusted

## References
<ul><li> <strong><a href="https://www.census.gov/retail/mrts/www/mrtssales92-present.xls">marts_source</a></strong>. Retail and Food Service Sales spreadsheet</li><ul>

In [3]:
s = pkg.reference('marts_source').url
s

'https://www.census.gov/retail/mrts/www/mrtssales92-present.xls'

In [4]:
def marts_split(url, sheet):
    
    
    df = pd.read_excel(s, sheet_name=sheet, skiprows=4)
    df.columns = ['naics','category'] + list(df.columns)[2:]

    
    groups = [ [], [] ] # Adjusted and Not adjusted
    group_n = None
    
    for idx, row in df.iterrows():
        if str(row.category).startswith('NOT ADJUSTED'):
            group_n = 0
        elif str(row.category).startswith('ADJUSTED'):
            group_n = 1
        elif isinstance(row.naics, str):
            try: # All all of the parts numbers?
                _ = [int(v) for v in str(row.naics).split(',')]
            except ValueError:
                group_n = None
            
            
            
        if group_n is not None:
            groups[group_n].append(row)
        
    adj = pd.DataFrame(groups[0]).set_index(['naics','category']).stack().to_frame('sales')
    nadj = pd.DataFrame(groups[1]).set_index(['naics','category']).stack().to_frame('sales')  
        
    return ( adj,nadj )
            




In [5]:
adj_frames = []
nadj_frames = []

for i in range(50):
    try:
        adj, nadj = marts_split(s, i)

        adj_frames.append(adj)
        nadj_frames.append(nadj)
    except IndexError:
        break
    


In [6]:
t = adj.copy()

t.head()

sales
naics category                                               
NaN   Retail and food services sales, total Jan. 1992  146376
                                            Feb. 1992  147079
                                            Mar. 1992  159336
                                            Apr. 1992  163669
                                            May 1992   170068

In [17]:
import datetime 
def parse_dates(v):
    try:
        return datetime.datetime.strptime(v.replace('(p)','').replace('.','')
                                    , '%b %Y')
    except ValueError:
        return None

def set_cypy(v):
    if 'CY' in v or 'TOTAL' in v:
        return 'CY'
    elif 'PY' in v:
        return 'PY'
    elif '(p)' in v:
        return 'PM'
    else:
        return 'M'
    
def set_cypy_date(r):
    if r.flag == 'CY' :
        return pd.Timestamp(f"{r.year}-01-01")
    elif r.flag == 'PY':
        return pd.Timestamp(f"{r.year-1}-01-01")

def post_process(t):
    
    t = t.reset_index().rename(columns={'level_2':'dt_str'})
    
    t['date'] = t.dt_str.apply(parse_dates)
    t['year'] = pd.to_numeric((t.date.dt.year).fillna(method='ffill')).astype(int)
    t['month'] = pd.to_numeric(t.date.dt.month, errors='coerce')
    t['flag'] = t.dt_str.apply(set_cypy)

    t['sales'] = pd.to_numeric(t.sales, errors='coerce')
    
    i  = t[t.date.isnull()].index

    t.loc[i, 'date'] = t.loc[i].apply(set_cypy_date, axis=1)
    
    return t

adj = post_process(pd.concat(adj_frames))
adj = adj[['naics', 'category', 'date',  'year', 'month',  'sales', 'flag']]
adj['sales'] = adj.sales * 1_000_000

nadj = post_process(pd.concat(nadj_frames))
nadj = nadj[['naics', 'category', 'date',  'year', 'month',  'sales', 'flag']]
nadj['sales'] = nadj.sales * 1_000_000
